## This example shows how to set up a Microservices Monorepo in GitHub

- The Monorepo contains two Node.js microservices using the Express Web Server packaged in Docker Images:
- - `video-streaming` that communicates with a Web Browser and the `video-storage` microservice.
  - `video-storage` that communicates with the `video-streaming` microservice and an Azure Storage Container.
- The `video-streaming` microservice has its own GitHub Actions Workflow in the Monorepo:
  - The microservice's GitHub Actions Workflow is only triggered on pushes to the microservices subfolder in the Monorepo, and.
    - Runs Jest unit tests.
    - Builds a Docker Container containing the microservice.
    - Pushes the Docker Container to an Azure COntainer Registry.
    - Deploys a Kubernetes YAML manifest file containing a Deployment and a LoadBalancer Service.
- The `video-storage` microservice has its own GitHub Actions Workflow in the Monorepo:
  - The microservice's GitHub Actions Workflow is only triggered on pushes to the microservices subfolder in the Monorepo, and.
    - Runs Jest unit tests.
    - Builds a Docker Container containing the microservice.
    - Pushes the Docker Container to an Azure COntainer Registry.
    - Deploys a Kubernetes YAML manifest file containing a Deployment and a ClusterIP Service.
- The deployed microservice application works as follows:
  - The URL `http://EXTERNAL-IP/video?id=SampleVideo_1280x720_1mb.mp4` is entered into a web browser.
    - The browser sends an HTTP GET request to the `video-streaming` microservice's GET route for the `/video` path.
    - The `video-streaming` microservice:
      - Extracts the `id` from the querystring.
      - Sends an HTTP GET request to the `video-storage` microservice's GET route for the `/video` path with `id=SampleVideo_1280x720_1mb.mp4`.
    - The `video-storage` microservice:
      - Extracts the `id` from the querystring.
      - Fetches the video from the Azure Storage Container `videos` by using the `id` to find the file.
    - The video is:
      - Streamed from the `video-storage` microservice to the `video-streaming` microservice.
      - Streamed from the `video-streaming` microservice to the web browser.
- Feel free to check the code for teh microservices:
  - The code for the `video-storage` microservice is in the file `video-storage/src/index.js`.
  - The code for the `video-streaming` microservice is in the file `video-streaming/src/index.js`.

## Login to GitHub via the CLI

### Run the command below in a separate terminal and follow the instructions

```bash
gh auth login --hostname github.com --git-protocol https --web
```

## Authorize running the Delete Repository Command

### Run the command below in a separate terminal and follow the instructions

```bash
gh auth refresh -h github.com -s delete_repo
```

## Change the Terraform variable `app_name`

- Open the file `variables.tf` in the subfolder `terraform`, and give your Terraform variable `app_name` a unique value.

  ```bash
  variable "app_name" {
    default = "tsfn14g00" # <--- change this value to something unique
  }
  ```

## Create a GitHub Repository

- These set of commands will:
  - Create a subfolder `monorepo` and copy over:
    - The `terraform` subfolder that contains Terraform `.tf` files for setting up resources in Azure:
      - Resource Group, Storage Account, Storage Container, Container Registry, Azure Kubernetes Service, Network Watcher.
    - The two subfolders for each microservice `video-storage` and `video-streaming`, where each contains:
      - A `kubernetes` subfolder with a kubernetes YAML manifest file `deploy.yaml` for deploying a Deployment and a Service.
      - A `src` subfolder with with the microservices code file `index.js` and a Jest test file `index.test.js`.
      - A `.dockerignore` and `.gitignore` file for the microservice.
      - A `Dockerfile` for the microservice.
      - A Jest config file `jest.config.js`.
      - A `package.json` and a `package-lock.json` file with information about the Node.js application and npm packages.
    - The `workflows` subfolder the two microservices' GitHub Actions Workflow files `video-storage.yaml` and `video-streaming.yaml`.
    - A `.gitignore` file for the Terraform files.
    - A `LICENSE` file.
    - A `README.md` file.
  - Run `git init`, `git all -A` and `git commit` in the `monorepo` subfolder.
  - Use the GitHub CLI tool `gh` to create a new GitHub Repository on your GitHub account from the `monorepo` subfolder.
- Check your GitHub account after running this cell, and you will find a new Repository called `monorepo`.

#### Run this on Linux/macOS

In [1]:
%%system

mkdir -p monorepo/.github
cp -r terraform video-storage video-streaming LICENSE README.md .gitignore monorepo
cp -r workflows monorepo/.github
cd monorepo

git init
git add -A
git commit --quiet -m "initial commit"

gh repo create monorepo \
--public \
--description "Microservices Monorepo" \
--source=. \
--remote=origin \
--push

cd ..

['Initialized empty Git repository in /home/patrick/projects/tsfn14/tsfn14/04_Azure_and_Github_Actions/monorepo/.git/',
 'https://github.com/paga-ju/monorepo',
 'To https://github.com/paga-ju/monorepo.git',
 ' * [new branch]      HEAD -> master',
 "Branch 'master' set up to track remote branch 'master' from 'origin'."]

#### Run this on Windows

In [1]:
%%system

mkdir -p monorepo\.github
xcopy terraform monorepo\ /E/H
xcopy video-storage monorepo\ /E/H
xcopy video-streaming monorepo\ /E/H
xcopy LICENSE monorepo\
xcopy README.md monorepo\
xcopy .gitignore monorepo\

cd monorepo

git init
git add -A
git commit --quiet -m "initial commit"

gh repo create monorepo ^
--public ^
--description "Microservices Monorepo" ^
--source=. ^
--remote=origin ^
--push

cd ..

['Initialized empty Git repository in /home/patrick/projects/tsfn14/tsfn14/04_Azure_and_Github_Actions/monorepo/.git/',
 'https://github.com/paga-ju/monorepo',
 'To https://github.com/paga-ju/monorepo.git',
 ' * [new branch]      HEAD -> master',
 "Branch 'master' set up to track remote branch 'master' from 'origin'."]

## Terraform Init and Terraform Apply

- Create the Azure infrastructure from the `terraform` folder inte `monorepo` subfolder.

In [2]:
%%system

cd monorepo/terraform

terraform init
terraform apply -auto-approve

cd ../..

['',
 '\x1b\x1bInitializing the backend...\x1b',
 '',
 '\x1b\x1bInitializing provider plugins...\x1b',
 '- Finding hashicorp/azurerm versions matching "3.90.0"...',
 '- Installing hashicorp/azurerm v3.90.0...',
 '- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)',
 '',
 'Terraform has created a lock file \x1b.terraform.lock.hcl\x1b to record the provider',
 'selections it made above. Include this file in your version control repository',
 'so that Terraform can guarantee to make the same selections by default when',
 'you run "terraform init" in the future.\x1b',
 '',
 '\x1b\x1b\x1bTerraform has been successfully initialized!\x1b\x1b\x1b',
 '\x1b\x1b',
 'You may now begin working with Terraform. Try running "terraform plan" to see',
 'any changes that are required for your infrastructure. All Terraform commands',
 'should now work.',
 '',
 'If you ever set or change modules or backend configuration for Terraform,',
 'rerun this command to reinitialize your working directory. 

## Get the Terraform variable `app_name` in the monorepo

- Store app_name in a Python variable so we can use it later in this notebook.

In [3]:
APP_NAME=!terraform -chdir=monorepo/terraform output
APP_NAME = APP_NAME[0].split('"')[1]
print(f"APP_NAME: {APP_NAME}")

APP_NAME: tsfn14g00


## 

## List Azure Resource Groups

- We see that two Azure Resource Groups were created from the `terraform` folder.
  - An additional Azure Resource Group was created by Azure, but will be deleted when we run `terraform destroy`.

In [4]:
!az group list -o table

Name                               Location    Status
---------------------------------  ----------  ---------
tsfn14g00                          westeurope  Succeeded
NetworkWatcherRG                   westeurope  Succeeded
MC_tsfn14g00_tsfn14g00_westeurope  westeurope  Succeeded


## List Azure Kubernetes Services

- We see that an Azure Kubernetes Service was created from the `terraform` folder.

In [5]:
!az aks list -o table

Name       Location    ResourceGroup    KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------  ----------  ---------------  -------------------  --------------------------  -------------------  -------------------------------------------
tsfn14g00  westeurope  tsfn14g00        1.27.7               1.27.7                      Succeeded            tsfn14g00-ddvzjt83.hcp.westeurope.azmk8s.io


## List Azure Container Registries

- We see that an Azure Container Registry was created from the `terraform` folder.

In [6]:
!az acr list -o table

NAME       RESOURCE GROUP    LOCATION    SKU    LOGIN SERVER          CREATION DATE         ADMIN ENABLED
---------  ----------------  ----------  -----  --------------------  --------------------  ---------------
tsfn14g00  tsfn14g00         westeurope  Basic  tsfn14g00.azurecr.io  2024-02-12T02:10:39Z  True


## List Azure Storage Accounts

- We see that an Azure Storage Account was created from the `terraform` folder.
  - It also creates an Storage Container called `videos` under the Azure Storage Account.

In [7]:
!az storage account list -o table

AccessTier    AllowBlobPublicAccess    AllowCrossTenantReplication    AllowSharedKeyAccess    CreationTime                      DefaultToOAuthAuthentication    EnableHttpsTrafficOnly    EnableNfsV3    IsHnsEnabled    IsSftpEnabled    Kind       Location    MinimumTlsVersion    Name       PrimaryLocation    ProvisioningState    PublicNetworkAccess    ResourceGroup    StatusOfPrimary
------------  -----------------------  -----------------------------  ----------------------  --------------------------------  ------------------------------  ------------------------  -------------  --------------  ---------------  ---------  ----------  -------------------  ---------  -----------------  -------------------  ---------------------  ---------------  -----------------
Hot           True                     True                           True                    2024-02-12T02:10:39.101446+00:00  False                           True                      False          False           False      

## Get Access Information for the Azure Container Registry

- We need this information to configure our microservices and kubernetes deployments.
- Let's store the LOGIN SERVER, USERNAME and PASSWORD in Python variables so we can use them later in this notebook.

In [8]:
CONTAINER_REGISTRY_LOGIN_SERVER=!az acr show -n {APP_NAME} --query loginServer -o tsv
CONTAINER_REGISTRY_LOGIN_SERVER=CONTAINER_REGISTRY_LOGIN_SERVER[0]
CONTAINER_REGISTRY_USERNAME=!az acr credential show -n {APP_NAME} --query username -o tsv
CONTAINER_REGISTRY_USERNAME=CONTAINER_REGISTRY_USERNAME[0]
CONTAINER_REGISTRY_PASSWORD=!az acr credential show -n {APP_NAME} --query passwords[0].value -o tsv
CONTAINER_REGISTRY_PASSWORD=CONTAINER_REGISTRY_PASSWORD[0]

print(f"CONTAINER_REGISTRY_LOGIN_SERVER: {CONTAINER_REGISTRY_LOGIN_SERVER}")
print(f"CONTAINER_REGISTRY_USERNAME: {CONTAINER_REGISTRY_USERNAME}")
print(f"CONTAINER_REGISTRY_PASSWORD: {CONTAINER_REGISTRY_PASSWORD}")

CONTAINER_REGISTRY_LOGIN_SERVER: tsfn14g00.azurecr.io
CONTAINER_REGISTRY_USERNAME: tsfn14g00
CONTAINER_REGISTRY_PASSWORD: IZDCtrY5gREP4T0aqt/Kjf/y7iIHEBroDCfzRX7tZJ+ACRBCfqQU


## Get Access Information for Azure Storage Account

- We need this information to configure our microservices and kubernetes deployments.
- Let's store the STORAGE ACCOUNT NAME and STORAGE ACCES KEY in Python variables so we can use them later in this notebook.

In [9]:
STORAGE_ACCOUNT_NAME=!az storage account list --query [0].name -o tsv
STORAGE_ACCOUNT_NAME=STORAGE_ACCOUNT_NAME[0]
STORAGE_ACCESS_KEY=!az storage account keys list --account-name tsfn14g00 --resource-group tsfn14g00 --query [0].value -o tsv
STORAGE_ACCESS_KEY=STORAGE_ACCESS_KEY[0]

print(f"STORAGE_ACCOUNT_NAME: {STORAGE_ACCOUNT_NAME}")
print(f"STORAGE_ACCESS_KEY: {STORAGE_ACCESS_KEY}")

STORAGE_ACCOUNT_NAME: tsfn14g00
STORAGE_ACCESS_KEY: ptIjvtN1ArJbTTdktZ2HXHYlxRWJY6sDDwiWj3QYt3mKa1ZPfIlJLRnZTF+//FZeBbK2Qved3KxH+AStQAwmVA==


## Upload a file to Container `videos` in Storage Container

- The microservices application doesn't have a front-end, so we can't upload a file to the Storage Container via a web interface.
- Here we manually upload the file `SampleVideo_1280x720_1mb.mp4` from the `videos` subfolder to the Storage Container.
  - We do this so the `video-storage` microservice will find the file when we test the applciation.

In [10]:
!az storage blob upload --container-name videos --name SampleVideo_1280x720_1mb.mp4 --file videos/SampleVideo_1280x720_1mb.mp4 --account-name {STORAGE_ACCOUNT_NAME} --account-key {STORAGE_ACCESS_KEY} -o table
!az storage blob list --container-name videos --account-name {STORAGE_ACCOUNT_NAME} --account-key {STORAGE_ACCESS_KEY} -o table 

Finished[#############################################################]  100.0000%
Client_request_id                     Content_md5               Date                       LastModified               Request_id                            Request_server_encrypted    Version
------------------------------------  ------------------------  -------------------------  -------------------------  ------------------------------------  --------------------------  ----------
ada885c0-c94c-11ee-ad81-fff0521565fd  1Vvd+NYpEIee2fYFUiFJqA==  2024-02-12T02:16:06+00:00  2024-02-12T02:16:07+00:00  29c30613-601e-004a-1759-5db42b000000  True                        2022-11-02
Name                          Blob Type    Blob Tier    Length    Content Type    Last Modified              Snapshot
----------------------------  -----------  -----------  --------  --------------  -------------------------  ----------
SampleVideo_1280x720_1mb.mp4  BlockBlob    Hot          1055736   video/mp4       2024-02-12T02:1

## Add Azure Kubernetes Cluster Info. to Local Kubectl Config File

- We need to fetch the configuration information from our Azure Kubernetes Service (AKS) and update our kubectl's config file.
- Here we are creating a backup of our current kubectl config file, before repalcing it will the AKS config file.
- Finally, we ensure we are using the AKS context when issuing kubectl commands.

**Note**

- The command below attaches the Azure Kubernetes Cluster to the Azure Container Registry
  - So that the Azure Kubernetes Cluster can pull images from the Azure Container Registry.
- But we don't have to do this here, since we do it in the Terraform file "kubernetes-cluster.tf".

  ```bash
  az aks update -n {APP_NAME} -g {APP_NAME} --attach-acr {APP_NAME} -o table
  ```

#### Run this on Linux/macOS

In [11]:
%%system

cp ~/.kube/config ~/.kube/config.bak
rm ~/.kube/config
az aks get-credentials --name {APP_NAME} --resource-group {APP_NAME}
kubectl config current-context

['WARNING: Merged "tsfn14g00" as current context in /home/patrick/.kube/config',
 'tsfn14g00']

#### Run this on Windows

In [102]:
%%system

xcopy %USERPROFILE%\.kube\config %USERPROFILE%\.kube\config.bak*
del %USERPROFILE%\.kube\config
az aks get-credentials --name {APP_NAME} --resource-group {APP_NAME}
kubectl config current-context

['WARNING: Merged "tsfn14g00" as current context in /home/patrick/.kube/config',
 'tsfn14g00']

## Get Base64 Encoded Kubectl Config File

- We need one final piece of information to configure our GitHub Actions workflow:
  - A Base64-encoded version of your kubectl config file.
  - This is needed by the GitHub Actions workflow running which will be issuing kubectl commands against the AKS cluster.
  - Let's store the Base64-encoded kubectl config file in a Python variable so we can use it later in this notebook.

**Note**

- On Windows, before running the cell below:
  - Download Base64 from here: https://www.di-mgt.com.au/base64-for-windows.html
  - Place the binary `Base64.exe` in the folder `04_Azure_and_Github_Actions`.
  - Run the command `KUBE_CONFIG=!type ~/.kube/config | base64 -w 0` instead of `KUBE_CONFIG=!cat ~/.kube/config | base64 -w 0`.

In [12]:
KUBE_CONFIG=!cat ~/.kube/config | base64 -w 0
KUBE_CONFIG=KUBE_CONFIG[0]

print(f"KUBE_CONFIG:\n{KUBE_CONFIG}")

KUBE_CONFIG:
YXBpVmVyc2lvbjogdjEKY2x1c3RlcnM6Ci0gY2x1c3RlcjoKICAgIGNlcnRpZmljYXRlLWF1dGhvcml0eS1kYXRhOiBMUzB0TFMxQ1JVZEpUaUJEUlZKVVNVWkpRMEZVUlMwdExTMHRDazFKU1VVMlZFTkRRWFJIWjBGM1NVSkJaMGxTUVZCQ1RWQm1hWG8zZEZoc2IyMUVTekJ2TVc1bVpUUjNSRkZaU2t0dldrbG9kbU5PUVZGRlRFSlJRWGNLUkZSRlRFMUJhMGRCTVZWRlFYaE5RMWt5UlhkSlFtTk9UV3BSZDAxcVJYbE5SRWwzVFZSSk0xZG9aMUJOYWtFeFRrUkJlVTFVU1hkTmFrVjRUV3BrWVFwTlFUQjRRM3BCU2tKblRsWkNRVTFVUVcxT2FFMUpTVU5KYWtGT1FtZHJjV2hyYVVjNWR6QkNRVkZGUmtGQlQwTkJaemhCVFVsSlEwTm5TME5CWjBWQkNqaHlNblZRVmpOT09VYzJlVkpFVWtwV1RFUnJZbEJaU1VadFFqaHFNM0pUYzFkWWFtNVhkVU5YUldaRWQxQm1TelpDYVdRM2RsUmtaekZ1WkVOb1EySUtVbFYwVkhoTlFWSkhkVWxQYlZkSmJteEhWR2gxTms0NGRFTjJjSFpKTUUxV05rNTBaamx4T0hsSmRYZE9Wbmh6WlV4alJESXpURFJzY0dOMlNtTTJVQW93ZUd4cmJrSXhjVmhTVkc5S1NHZE1jMHN2Ykhjd1VFTnJhVFZPTm5sc2VsVXZNRWhWU0hGWVNpOXBPR2sxTlhGNE1YbEpWVE01YVVzclFWQnhTbUpSQ2xoUU5qTjBORVUxWTFGcmEyTTROREIyZUhKT1JWQmtVVU50WTBkRmRYQjFhalZhWjJoNmJEUmFZVGgzWjNnck5ub3paMlJCTkd0WFRuRTJURUZCYWxBS1lYSTRVa0Z3WWxwaFUxVkRkMlY0Vms5NU1FbHF

## Set GitHub Actions Repository Secrets

- We can add "secret" key-value pairs to our GitHub Repository, which can be accessed by our GitHub Actions Workflows.
- We do this so we don't hard-code sensitive information into our GitHub Actions Workflow files.
- You can do this manually in GitHub by:
 - Clicking on your `monorepo` Respository in GitHub.
 - Clicking on the `Settings` tab near the top of the web page.
 - Expanding the `Secrets and Values` combobox in the left margin on the web page and choosing `Actions`.
 - Clicking the button `New repository secret` under `Repository secrets`.
 - Entering a `Name` and a `Secret`, followed by clicking `Add secret`.
 - Repeat the previous step for all the necesary key-value pairs for the keys.
   - `CONTAINER_REGISTRY_LOGIN_SERVER`
   - `CONTAINER_REGISTRY_USERNAME`
   - `CONTAINER_REGISTRY_PASSWORD`
   - `STORAGE_ACCOUNT_NAME`
   - `STORAGE_ACCESS_KEY`
   - `KUBE_CONFIG`
- The values (secrets) you need to enter for these keys (names) are repeated below.

In [13]:
print(f"CONTAINER_REGISTRY_LOGIN_SERVER: {CONTAINER_REGISTRY_LOGIN_SERVER}")
print(f"CONTAINER_REGISTRY_USERNAME: {CONTAINER_REGISTRY_USERNAME}")
print(f"CONTAINER_REGISTRY_PASSWORD: {CONTAINER_REGISTRY_PASSWORD}")
print(f"STORAGE_ACCOUNT_NAME: {STORAGE_ACCOUNT_NAME}")
print(f"STORAGE_ACCESS_KEY: {STORAGE_ACCESS_KEY}")
print(f"KUBE_CONFIG: {KUBE_CONFIG}")

CONTAINER_REGISTRY_LOGIN_SERVER: tsfn14g00.azurecr.io
CONTAINER_REGISTRY_USERNAME: tsfn14g00
CONTAINER_REGISTRY_PASSWORD: IZDCtrY5gREP4T0aqt/Kjf/y7iIHEBroDCfzRX7tZJ+ACRBCfqQU
STORAGE_ACCOUNT_NAME: tsfn14g00
STORAGE_ACCESS_KEY: ptIjvtN1ArJbTTdktZ2HXHYlxRWJY6sDDwiWj3QYt3mKa1ZPfIlJLRnZTF+//FZeBbK2Qved3KxH+AStQAwmVA==
KUBE_CONFIG: YXBpVmVyc2lvbjogdjEKY2x1c3RlcnM6Ci0gY2x1c3RlcjoKICAgIGNlcnRpZmljYXRlLWF1dGhvcml0eS1kYXRhOiBMUzB0TFMxQ1JVZEpUaUJEUlZKVVNVWkpRMEZVUlMwdExTMHRDazFKU1VVMlZFTkRRWFJIWjBGM1NVSkJaMGxTUVZCQ1RWQm1hWG8zZEZoc2IyMUVTekJ2TVc1bVpUUjNSRkZaU2t0dldrbG9kbU5PUVZGRlRFSlJRWGNLUkZSRlRFMUJhMGRCTVZWRlFYaE5RMWt5UlhkSlFtTk9UV3BSZDAxcVJYbE5SRWwzVFZSSk0xZG9aMUJOYWtFeFRrUkJlVTFVU1hkTmFrVjRUV3BrWVFwTlFUQjRRM3BCU2tKblRsWkNRVTFVUVcxT2FFMUpTVU5KYWtGT1FtZHJjV2hyYVVjNWR6QkNRVkZGUmtGQlQwTkJaemhCVFVsSlEwTm5TME5CWjBWQkNqaHlNblZRVmpOT09VYzJlVkpFVWtwV1RFUnJZbEJaU1VadFFqaHFNM0pUYzFkWWFtNVhkVU5YUldaRWQxQm1TelpDYVdRM2RsUmtaekZ1WkVOb1EySUtVbFYwVkhoTlFWSkhkVWxQYlZkSmJteEhWR2gxTms0NGRFTjJjSFpKTUUxV05rNTBaamx

## Set the key-value (name-secret) pairs using the GitHub CLI

- Alternatively, we can set the GitHub Actions Name-Secret pairs using the GitHub CLI below.

In [14]:
%%system

cd monorepo

gh secret set CONTAINER_REGISTRY_LOGIN_SERVER --body {CONTAINER_REGISTRY_LOGIN_SERVER}
gh secret set CONTAINER_REGISTRY_USERNAME --body {CONTAINER_REGISTRY_USERNAME}
gh secret set CONTAINER_REGISTRY_PASSWORD --body {CONTAINER_REGISTRY_PASSWORD}
gh secret set STORAGE_ACCOUNT_NAME --body {STORAGE_ACCOUNT_NAME}
gh secret set STORAGE_ACCESS_KEY --body {STORAGE_ACCESS_KEY}
gh secret set KUBE_CONFIG --body {KUBE_CONFIG}

gh secret list

cd ..

['CONTAINER_REGISTRY_LOGIN_SERVER\t2024-02-12T02:48:43Z',
 'CONTAINER_REGISTRY_PASSWORD\t2024-02-12T02:48:45Z',
 'CONTAINER_REGISTRY_USERNAME\t2024-02-12T02:48:44Z',
 'KUBE_CONFIG\t2024-02-12T02:48:46Z',
 'STORAGE_ACCESS_KEY\t2024-02-12T02:48:46Z',
 'STORAGE_ACCOUNT_NAME\t2024-02-12T02:48:45Z']

## List GitHub Actions Workflows in Repository

- Let's use the GitHub CLI to list the GitHub Actions Workflows we have in our `monorepo` repository.
- We see two Workflows, one for each of our two microservices.
- If you look at the workflow for the `video-storage` microservice, you will see:
  - How the "secrets" we just added are accessed in the workflow, e.g. `secrets.CONTAINER_REGISTRY_LOGIN_SERVER`.
  - How the `paths` setting is used to only trigger the workflow when a `git push` is make to the `video-storage`subfolder:

    ```bash
    push:
        branches:
          - master
        paths:
          - 'video-storage/**' # Only trigger on push to video-storage subfolder.
    ```

  - How the `working-directory` setting is used to make all `run` commands execute in the `video-storage` subfolder:
  - 
    ```bash
    jobs:
      video-storage:
        runs-on: ubuntu-latest
        defaults:
          run:
            working-directory: 'video-storage' # Sets default path for "run" commands to the video-storage subfolder.
    ```

  - How the `cache-dependency-path` setting is used to reference the `package-lock.json` file in the `video-streaming` subfolder:
  - 
    ```bash
    # Installs Node.js.
          - uses: actions/setup-node@v4
            with:
              node-version: 19.9.0
              cache: 'npm'
              cache-dependency-path: 'video-storage/package-lock.json' # Path to package-lock.json in video-storage.
    ```
  - How the hash SHA (Secure Hash Algoritm) is used for the latest commit to set the version for the Docker Image:

    ```bash
    VERSION: ${{ github.sha }} # Quick and dirty way to automatically set the next version for the Docker Image
    ```

In [15]:
%%system

cd monorepo
gh workflow list
cd ..

['Deploy video-storage microservice\tactive\t85694508',
 'Deploy video-streaming microservice\tactive\t85694509']

## Run GitHub Actions Workflows in Repository

- Let's trigger our first run of the two workflows manually.
- After running the cell below, check the workflows in action on GitHub, where each workflow will:
  - Run Jest tests for the microservice.
  - Build a Docker Image for the microservice (with a new version tag).
  - Push the Docker image to the Azure Container Registry.
  - Deploy the microservice's YAML manifest file using kubectl.
- Only continue to the next cell once the GitHub Actions Workflows have completed.

In [16]:
%%system

cd monorepo
gh workflow run "Deploy video-storage microservice"
gh workflow run "Deploy video-streaming microservice"
cd ..

[]

## Get Load Balancer Public IP Address

- The `video-streaming` microservice has a kubenetes Service of type Loadbalancer.
- Let's get the Service's public IP address so we can access teh kubernetes cluster over the internet.
- Let's store the Load Balancer's EXTERNAL-IP (public IP) in a Python variable so we can use it later in this notebook.

In [17]:
LOAD_BALANCER_PUBLIC_IP=!kubectl get service video-streaming -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
LOAD_BALANCER_PUBLIC_IP=LOAD_BALANCER_PUBLIC_IP[0]

print(f"LOAD_BALANCER_PUBLIC_IP: {LOAD_BALANCER_PUBLIC_IP}")

LOAD_BALANCER_PUBLIC_IP: 20.126.192.254


## Test the Microservice Application in the Kubernetes Cluster

- Open a browser and enter the URL `http://EXTERNAL-IP/video?id=SampleVideo_1280x720_1mb.mp4`, where `EXTERNAL-IP` is the IP address above.
  - The browser sends an HTTP GET request to the video-streaming microservice's GET route for the `/video` path.
  - The video-streaming microservice:
    - Extracts the `id` from the querystring.
    - Sends an HTTP GET request to the video-storage microservice's GET route for the `/video` path with `id=SampleVideo_1280x720_1mb.mp4`.
  - The video-storage microservice:
    - Extracts the `id` from the querystring.
    - Fetches the video from the Azure Storage Container `videos` by using the `id` to find the file.
  - The video is:
    - Streamed from the video-storage microservice to the video-streaming microservice.
    - Streamed from the video-streaming microservice to the web browser.
- You should see a video being played in your broswer.

In [18]:
#!firefox http://{LOAD_BALANCER_PUBLIC_IP}/video?id=SampleVideo_1280x720_1mb.mp4

[GFX1-]: glxtest: ManageChildProcess failed



## Get image name of current video-streaming Pod

- Let's get the name of the Docker image used by the `video-streaming` Pod.
- Notice the version tag at the very end on the image name.
  - This is the hash (SHA) automatically set by the GitHub Actions Workflow.

In [19]:
!kubectl get deployment video-streaming -o=jsonpath='{$.spec.template.spec.containers[0].image}'

tsfn14g00.azurecr.io/video-streaming:a17ea8f0fce63e50b70d7c9b7312a054adfeb260

## Modify file in video-streamning microservice and push changes to GitHub

- Now, let's change the file `index.js` for the `video-streaming` microservice in the `monorepo` repository.
  - Followed by a `git add`, a `git commit` and a `git push`.
- Since we are chaning a file in the `video-streaming` subfolder, only the `video-streaming` GitHub Actions Workflow will be triggered.
- This will result in a new Docker image, with a new version tag, being deployed to the Container Registry.
  - Followed by a new `kubectl apply` to the updated YAML manifest file for the `video-streaming` microservice.
- After running the cell below, make sure the GitHub Actions `video-streaming` Workflow on GitHub has completed before continuing.
- **Note**: On Windows, run the command `echo // Comment >> video-streaming\src\index.js`
  - instead of `echo '// Comment' >> video-streaming/src/index.js`

In [20]:
%%system

cd monorepo

echo '// Comment' >> video-streaming/src/index.js
git add video-streaming/src/index.js
git commit -m "Update index.js"
git push

cd ..

['[master f66cf21] Update index.js',
 ' 1 file changed, 1 insertion(+), 1 deletion(-)',
 'To https://github.com/paga-ju/monorepo.git',
 '   a17ea8f..f66cf21  master -> master']

## Get image name of current video-streaming Pod

- Let's get the name of the Docker image used by the `video-streaming` Pod again.
- Notice the version tag at the very end on the image name.
  - We see a new hash (SHA) version tag value for the Docker image.

In [21]:
!kubectl get deployment video-streaming -o=jsonpath='{$.spec.template.spec.containers[0].image}'

tsfn14g00.azurecr.io/video-streaming:f66cf21591395153bc8a054ebe6037649a09805f

## Ensure the Kubectl Context is set to the Minikube Cluster

- Let's restore our local kubectl config file.
- The cell below will replace the existing kubectl config file with the backup we made before.
  - Followed by changing the context to `minikube`.
    - If you aren't using minikube, use `kubectl config unset current-context` below instead of `kubectl config use-context minikube`.
- **Note**: On Windows, run the command `xcopy %USERPROFILE%\.kube\config.bak %USERPROFILE%\.kube\config*`
  - instead of `cp ~/.kube/config.bak ~/.kube/config`.

In [22]:
%%system

cp ~/.kube/config.bak ~/.kube/config
kubectl config use-context minikube
kubectl config current-context

['Switched to context "minikube".', 'minikube']

## Terraform Destroy

- Let's destroy our infrastructure on Azure by running `terraform destroy` in the `terraform` folder in the `monorepo` subfolder.

In [23]:
%%system

cd monorepo/terraform
terraform destroy -auto-approve
cd ../..

['\x1b\x1bazurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]\x1b',
 '\x1b\x1bazurerm_resource_group.networkwatcher: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/NetworkWatcherRG]\x1b',
 '\x1b\x1bazurerm_container_registry.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerRegistry/registries/tsfn14g00]\x1b',
 '\x1b\x1bazurerm_storage_account.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.Storage/storageAccounts/tsfn14g00]\x1b',
 '\x1b\x1bazurerm_kubernetes_cluster.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00/providers/Microsoft.ContainerService/managedClusters/tsfn14g00]\x1b',
 '\x1b\x1bazurerm_network_watcher.networkwatcher: Refreshing st

## Delete GitHub Repository

- Now we can delete the `monorepo` GitHub repository and the `monorepo` subfolder to clean things up.
- **Note**: On WIndows, run the command `rmdir /s /q monorepo` instead of `rm -rf monorepo`.

In [24]:
%%system

cd monorepo
gh repo delete monorepo --yes
cd ..

rm -rf monorepo

[]

## List Azure Storage Accounts

- We see the Azure Storage Account has been destroyed.

In [25]:
!az storage account list -o table

## List Azure Container Registries

- We see the Azure Container Registry has been destroyed.

In [26]:
!az acr list -o table

## List Azure Kubernetes Services

- We see the Azure Kubernetes Service has been destroyed.

In [27]:
!az aks list -o table

## List Azure Resource Groups

- We see the Azure Resource Groups have been destroyed.

In [28]:
!az group list -o table